In [1]:
import json

data_path = "../data/VLSP 2025 - MLQA-TSR Data Release/train_data/vlsp_2025_train.json"
with open(data_path, "r") as f:
    data = json.load(f)

print(data[0:10])



[{'id': 'train_1', 'image_id': 'train_1_3', 'question': 'Biển báo cấm xe khách trên 29 chỗ được áp dụng trong các khoảng thời gian nào? ', 'relevant_articles': [{'law_id': 'QCVN 41:2024/BGTVT', 'article_id': '22'}], 'question_type': 'Multiple choice', 'choices': {'A': 'Từ 6:30 đến 8:00 và từ 16:30 đến 18:30; ngoài các khoảng thời gian này không được phép lưu thông.', 'B': 'Từ 6:30 đến 8:00 và từ 16:30 đến 18:30; ngoài các khoảng thời gian này được phép lưu thông.', 'C': 'Cấm lưu thông cả ngày.', 'D': 'D. Không cấm xe khách trên 29 chỗ lưu thông.'}, 'answer': 'B'}, {'id': 'train_2', 'image_id': 'train_1_3', 'question': 'Những loại phương tiện nào bị cấm trên đoạn đường này', 'relevant_articles': [{'law_id': 'QCVN 41:2024/BGTVT', 'article_id': '26'}, {'law_id': 'QCVN 41:2024/BGTVT', 'article_id': '14'}], 'question_type': 'Multiple choice', 'choices': {'A': 'Xe khách trên 29 chỗ', 'B': 'ô tô con', 'C': 'Xe máy', 'D': 'Xe đạp'}, 'answer': 'A'}, {'id': 'train_3', 'image_id': 't

In [2]:
for record in data:
    if record["question_type"] == "Yes/No":
        answer = record.pop("answer")
        record["choices"] = None
        record["answer"] = answer
    file_name = f"{record["image_id"]}.jpg"
    record["file_name"] = file_name

In [3]:
print(data[0].keys())

dict_keys(['id', 'image_id', 'question', 'relevant_articles', 'question_type', 'choices', 'answer', 'file_name'])


In [4]:
import csv
df = open("../data/metadata.csv", "w")
writer = csv.writer(df)
for idx, data_row in enumerate(data):
    if idx==0:
        header = data_row.keys()
        writer.writerow(header)
    writer.writerow(data_row.values())
df.close()

In [5]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="../data/VLSP 2025 - MLQA-TSR Data Release/train_data/train_images")

/home/nhhsag12/Documents/Multimodal Machine Learning/VLSP_2025/vlsp_2025_multimodal_rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 530 examples [00:00, 5354.22 examples/s]


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'image_id', 'question', 'relevant_articles', 'question_type', 'choices', 'answer', 'image'],
        num_rows: 530
    })
})

In [7]:
dataset.push_to_hub("nhhsag12/vlsp_2025_multimodal_rag", private=True)


Map: 100%|██████████| 530/530 [00:00<00:00, 635.82 examples/s]

Uploading the dataset shards: 100%|██████████| 1/1 [00:10<00:00, 10.25s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/nhhsag12/vlsp_2025_multimodal_rag/commit/65c7f4966ad691ff792709005c8caa1ec4bab7c2', commit_message='Upload dataset', commit_description='', oid='65c7f4966ad691ff792709005c8caa1ec4bab7c2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/nhhsag12/vlsp_2025_multimodal_rag', endpoint='https://huggingface.co', repo_type='dataset', repo_id='nhhsag12/vlsp_2025_multimodal_rag'), pr_revision=None, pr_num=None)